In [ ]:
import sys
import heapq

INF = 10**18

def dijkstra(n, adj, src):
    dist = [INF] * (n + 1)
    dist[src] = 0
    pq = [(0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]:
            continue
        for v, w in adj[u]:
            nd = d + w
            if nd < dist[v]:
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return dist


def main():
    N, M, K, S, F = map(int, input().split())
    adj = [[] for _ in range(N + 1)]
    req = []
    for _ in range(M):
        u, v, w = map(int, input().split())
        adj[u].append((v, w))
        adj[v].append((u, w))
    for _ in range(min(K, 1)):
        req = list(map(int, input().split()))

    specials = [S] + req + [F]
    P = len(specials)
    dist_from = [dijkstra(N, adj, node) for node in specials]

    dist = [[INF]*P for _ in range(P)]
    for i in range(P):
        for j in range(P):
            dist[i][j] = dist_from[i][specials[j]]

    if K == 0:
        d = dist[0][1]
        print(d)
        print(S, specials[1])
        return

    FULL = (1 << K) - 1
    dp = [[INF] * K for _ in range(1 << K)]
    parent = [[(-1, -1)] * K for _ in range(1 << K)]

    for i in range(K):
        d = dist[0][i + 1]
        dp[1 << i][i] = d
        parent[1 << i][i] = (0, -1)

    for mask in range(1 << K):
        for last in range(K):
            if not (mask & (1 << last)): 
                continue
            cur = dp[mask][last]
            if cur >= INF: 
                continue
            for nxt in range(K):
                if mask & (1 << nxt): 
                    continue
                nd = cur + dist[last+1][nxt+1]
                nmask = mask | (1 << nxt)
                if nd < dp[nmask][nxt]:
                    dp[nmask][nxt] = nd
                    parent[nmask][nxt] = (mask, last)

    best = INF
    end_idx = -1
    for last in range(K):
        val = dp[FULL][last] + dist[last + 1][K + 1]
        if val < best:
            best = val
            end_idx = last

    order = []
    mask = FULL
    cur = end_idx
    while cur != -1:
        order.append(cur)
        pmask, pidx = parent[mask][cur]
        mask, cur = pmask, pidx
    order.reverse()

    seq = [str(S)] + [str(req[i]) for i in order] + [str(F)]
    print(best)
    print(' '.join(seq))


if __name__ == "__main__":
    main()